In [2]:
import pandas as pd

footfall_df= pd.read_csv("footfall_history.csv")
menu_df= pd.read_csv("menu.csv")
weather_df= pd.read_csv("weather.csv")
calendar_df= pd.read_csv("calendar.csv")


footfall_df.info()
menu_df.info()
weather_df.info()
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       12000 non-null  object
 1   time       12000 non-null  object
 2   headcount  12000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 281.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         360 non-null    object
 1   meal         360 non-null    object
 2   main_course  360 non-null    object
dtypes: object(3)
memory usage: 8.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          12000 non-null  object 
 1   time          12000 non-null  object 
 2   temp_c     

In [3]:
footfall_df.head()

,date,time,headcount
0,2025-01-15,18:00,0
1,2025-01-15,18:01,0
2,2025-01-15,18:02,10
3,2025-01-15,18:03,9
4,2025-01-15,18:04,4


In [4]:
menu_df.head()

,date,meal,main_course
0,2025-01-15,Breakfast,Poha
1,2025-01-15,Lunch,Rajma Chawal
2,2025-01-15,Dinner,Paneer Butter Masala
3,2025-01-16,Breakfast,Pancakes
4,2025-01-16,Lunch,Dal Fry


In [5]:
weather_df.head()

,date,time,temp_c,humidity_pct
0,2025-01-15,00:00,21.0,83
1,2025-01-15,00:15,22.3,78
2,2025-01-15,00:30,21.9,79
3,2025-01-15,00:45,24.6,77
4,2025-01-15,01:00,23.3,77


In [6]:
calendar_df.head()

,date,event
0,2025-01-15,Regular Classes
1,2025-01-16,Regular Classes
2,2025-01-17,Tech Fest
3,2025-01-18,Regular Classes
4,2025-01-19,Regular Classes


In [7]:
footfall_df['datetime']= pd.to_datetime(footfall_df['date'] + ' ' + footfall_df['time'])
evening_mask = footfall_df['datetime'].dt.hour.between(19, 20)  
evening_footfall = footfall_df[evening_mask]

peak_headcount= (
    evening_footfall.groupby('date')['headcount']
    .max()
    .reset_index()
)

peak_headcount.head()


,date,headcount
0,2025-01-15,15
1,2025-01-16,15
2,2025-01-17,18
3,2025-01-18,12
4,2025-01-19,11


In [8]:
weather_df['datetime']= pd.to_datetime(weather_df['date']+ ' '+ weather_df['time'])
evening_weather= weather_df[weather_df['datetime'].dt.hour.between(16, 20)] 
# bw 4 and 9 pm

weather_avg= (
    evening_weather.groupby('date')[['temp_c', 'humidity_pct']]
    .mean()
    .reset_index()
)

weather_avg.head()


,date,temp_c,humidity_pct
0,2025-01-15,12.765,60.90
1,2025-01-16,12.770,61.10
2,2025-01-17,12.795,61.10
3,2025-01-18,12.905,62.10
4,2025-01-19,12.970,62.15


In [9]:
dinner_menu= menu_df[menu_df['meal']== 'Dinner']
dishes_encoded= dinner_menu.explode('main_course')
dishes_encoded['main_course']= dishes_encoded['main_course'].str.strip()

one_hot= pd.get_dummies(dishes_encoded['main_course'])
dinner_encoded= pd.concat([dishes_encoded[['date']], one_hot],axis=1)

menu_features = dinner_encoded.groupby('date').sum().reset_index()
menu_features.head()

,date,Chole Bhature,Egg Curry,Kofta Curry,Paneer Butter Masala,Veg Biryani
0,2025-01-15,0,0,0,1,0
1,2025-01-16,0,0,0,1,0
2,2025-01-17,0,1,0,0,0
3,2025-01-18,0,0,0,0,1
4,2025-01-19,0,0,0,1,0


In [10]:

event_hot = calendar_df.explode('event')

event_encoded = pd.get_dummies(event_hot['event'])
event_encoded = pd.concat([event_hot[['date']], event_encoded], axis=1)

event_features = event_encoded.groupby('date').sum().reset_index()

event_features.head()


,date,Festival,Institute Holiday,Minor Exams,Regular Classes,Sports Day,Tech Fest
0,2025-01-15,0,0,0,1,0,0
1,2025-01-16,0,0,0,1,0,0
2,2025-01-17,0,0,0,0,0,1
3,2025-01-18,0,0,0,1,0,0
4,2025-01-19,0,0,0,1,0,0


In [11]:
df= pd.merge(peak_headcount, weather_avg, on='date')
df= pd.merge(df, menu_features, on='date', how='left')
df= pd.merge(df, event_features, on='date', how='left')


In [12]:
df= df.dropna()
df.head()

,date,headcount,temp_c,humidity_pct,Chole Bhature,Egg Curry,Kofta Curry,Paneer Butter Masala,Veg Biryani,Festival,Institute Holiday,Minor Exams,Regular Classes,Sports Day,Tech Fest
0,2025-01-15,15,12.765,60.90,0,0,0,1,0,0,0,0,1,0,0
1,2025-01-16,15,12.770,61.10,0,0,0,1,0,0,0,0,1,0,0
2,2025-01-17,18,12.795,61.10,0,1,0,0,0,0,0,0,0,0,1
3,2025-01-18,12,12.905,62.10,0,0,0,0,1,0,0,0,1,0,0
4,2025-01-19,11,12.970,62.15,0,0,0,1,0,0,0,0,1,0,0


In [13]:
X= df.drop(columns=['date', 'headcount'])
y= df['headcount']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [25]:
from sklearn.metrics import mean_absolute_error, r2_score

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

model= RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred= model.predict(X_test)
mae= mean_absolute_error(y_test, y_pred)
r2= r2_score(y_test, y_pred)

print(mae)
print(r2)

2.9662500000000005
-0.08847622384069331


In [29]:
from sklearn.linear_model import LinearRegression

model1= LinearRegression()
model1.fit(X_train, y_train)

y_pred1= model1.predict(X_test)
mae1= mean_absolute_error(y_test, y_pred1)
r21= r2_score(y_test, y_pred1)

print(mae1)
print(r21)


2.868131293998409
0.03813205125868768


In [32]:
from sklearn.tree import DecisionTreeRegressor

model2= DecisionTreeRegressor(random_state=42)
model2.fit(X_train, y_train)

y_pred2= model2.predict(X_test)
mae2= mean_absolute_error(y_test, y_pred2)
r22= r2_score(y_test, y_pred2)

print(mae2)
print(r22)

3.8333333333333335
-0.6461694477735778


In [34]:
from sklearn.ensemble import GradientBoostingRegressor

model3= GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model3.fit(X_train, y_train)

y_pred3= model3.predict(X_test)
mae3= mean_absolute_error(y_test, y_pred3)
r23= r2_score(y_test, y_pred3)

print(mae3)
print(r23)

3.3919290565296776
-0.3110443405180068


In [33]:
from xgboost import XGBRegressor

model4= XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model4.fit(X_train, y_train)

y_pred4= model4.predict(X_test)
mae4= mean_absolute_error(y_test, y_pred4)
r24= r2_score(y_test, y_pred4)

print(mae4)
print(r24)

3.0233392318089805
-0.2793863448063556


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model5= KNeighborsRegressor(n_neighbors=5)
model5.fit(X_train, y_train)

y_pred5= model5.predict(X_test)
mae5= mean_absolute_error(y_test, y_pred5)
r25= r2_score(y_test, y_pred5)

print(mae5)
print(r25)

2.858333333333333
0.029025537014843006


In [35]:
import joblib

joblib.dump(model1, 'model.pkl')
joblib.dump(X.columns.tolist(), 'features.pkl')

['features.pkl']